In [26]:
import riak

client = riak.RiakClient()
bucket = client.bucket("bucket_name")



In [27]:
bucket.new(key="key_name", encoded_data='2',
               content_type='application/octet-stream') \
        .add_link(bucket.new("foo1")) \
        .add_link(bucket.new("foo2"), "tag") \
        .add_link(bucket.new("foo3"), "tag2!@#%^&*)") \
        .store()



In [28]:
from test_helper import Test

obj = bucket.get("key_name")
links = obj.links

Test.assertEquals(len(links), 3)
for bucket, key, tag in links:
    if (key == "foo1"):
        Test.assertEquals(bucket, "bucket_name")
    elif (key == "foo2"):
        Test.assertEquals(tag, "tag")
    elif (key == "foo3"):
        Test.assertEquals(tag, "tag2!@#%^&*)")
    else:
        Test.assertEquals(key, "unknown key")

1 test passed.
1 test passed.
1 test passed.
1 test passed.


In [31]:
!curl -I http://localhost:8098/riak/bucket_name/key_name

In [32]:
!curl -I http://localhost:8098/riak/bucket_name/foo2

In [34]:
# Create the object
bucket = client.bucket("bucket_name")
o = bucket.new("key_name2", 2)
o.links = [("bucket_name", "foo1", None),
           ("bucket_name", "foo2", "tag"),
           ("bucket", "foo2", "tag2")]
o.store()
obj = bucket.get("key_name2")
links = sorted(obj.links, key=lambda x: x[1])
Test.assertEquals(len(links), 3)
Test.assertEquals(links[0][1], "foo1")
Test.assertEquals(links[1][1], "foo2")
Test.assertEquals(links[1][2], "tag")
Test.assertEquals(links[2][1], "foo2")
Test.assertEquals(links[2][2], "tag2")

1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.
1 test passed.


In [35]:
!curl -I http://localhost:8098/riak/bucket_name/key_name2

test_erlang_map_reduce

In [41]:
bucket = client.bucket("bucket_name3")
bucket.new("foo", 2).store()
bucket.new("bar", 2).store()
bucket.new("baz", 4).store()
# Run the map...
result = client \
    .add("bucket_name3", "foo") \
    .add("bucket_name3", "bar") \
    .add("bucket_name3", "baz") \
    .map(["riak_kv_mapreduce", "map_object_value"]) \
    .reduce(["riak_kv_mapreduce", "reduce_set_union"]) \
    .run()
    
Test.assertEquals(len(result), 2)

1 test passed.


test_erlang_map_reduce_bucket_type

In [43]:
btype = client.bucket_type("pytest")
bucket = btype.bucket("bucket_name4")
bucket.new("foo", 2).store()
bucket.new("bar", 2).store()
bucket.new("baz", 4).store()

# Run the map...
result = client \
    .add("bucket_name4", "foo", bucket_type="pytest") \
    .add("bucket_name4", "bar", bucket_type="pytest") \
    .add"bucket_name4", "baz", bucket_type="pytest") \
    .map(["riak_kv_mapreduce", "map_object_value"]) \
    .reduce(["riak_kv_mapreduce", "reduce_set_union"]) \
    .run()

Test.assertEquals(len(result), 2)

RiakError: 'Invalid bucket type: <<"pytest">>'

In [44]:
https://github.com/basho/riak-python-client/blob/master/riak/tests/test_mapreduce.py

SyntaxError: invalid syntax (<ipython-input-44-c7fe5d9bc9e5>, line 1)